In [2]:
!pip -q install sentence-transformers faiss-cpu rank-bm25 pandas numpy tqdm pydantic==2.* rapidfuzz

zsh:1: no matches found: pydantic==2.*


In [3]:
import os, json, re, ast, glob, uuid
from dataclasses import dataclass
from typing import List, Dict, Any, Optional
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
import faiss
from rapidfuzz import fuzz

ModuleNotFoundError: No module named 'numpy'

In [ ]:
import os, glob, unicodedata

# === 경로 ===
DATA_ROOT = "/content/data"
OUT_DIR   = "/content/artifacts"     # 인덱스/메타데이터 저장 폴더
os.makedirs(OUT_DIR, exist_ok=True)

print("DATA_ROOT =", DATA_ROOT)
print("OUT_DIR   =", OUT_DIR)

# 폴더/파일 간단 점검
def _ls(p):
    try:
        print("->", p);
        for x in sorted(os.listdir(p)):
            print("  ", x)
    except Exception as e:
        print("  (확인 실패)", e)

_ls(DATA_ROOT)
_ls(os.path.join(DATA_ROOT, "resumes"))
_ls(os.path.join(DATA_ROOT, "questions"))
_ls(os.path.join(DATA_ROOT, "jd"))

# === 인재상 파일 자동 탐색 ===
def find_values_path(root="/content/data"):
    # 1) 단순 glob 매칭
    cands = glob.glob(os.path.join(root, "*재상*.txt"))
    if cands:
        return cands[0]

    # 2) 정규화(NFC) 맞춰서 비교
    target = unicodedata.normalize("NFC", "인재상.txt")
    for name in os.listdir(root):
        if unicodedata.normalize("NFC", name) == target:
            return os.path.join(root, name)

    return None

values_path = find_values_path(DATA_ROOT)
print("values_path =", values_path, "| exists?", os.path.exists(values_path) if values_path else None)

DATA_ROOT = /content/data
OUT_DIR   = /content/artifacts
-> /content/data
   .ipynb_checkpoints
   jd
   questions
   resumes
   인재상.txt
-> /content/data/resumes
   자소서1_cleaned.txt
   자소서1_masked.txt
   자소서2_cleaned.txt
   자소서2_masked.txt
-> /content/data/questions
   Design_unique_questions.csv
   ICT_unique_questions.csv
   Management__unique_questions.csv
   PM_unique_questions.csv
   PublicService_unique_questions.csv
   RND_unique_questions.csv
   SalesMarketing_unique_questions.csv
-> /content/data/jd
   .ipynb_checkpoints
   Carrot_JD.txt
   Coupang_JD.txt
   KAKAO_Design_Staff_JD.txt
   KAKAO_ServiceBiz_JD.txt
   KAKAO_TECH_JD.txt
   LINE_Corporate_JD.txt
   LINE_DesignBSMarketing_JD.txt
   LINE_Engineering_JD.txt
   LINE_PlanningAnalysis_JD.txt
   NAVER_TECH_JD.txt
   Toss_JD.txt
   Woowahan_JD.txt
values_path = /content/data/인재상.txt | exists? True


In [ ]:
def read_text(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def literal_dict_from_txt(path: str) -> Dict[str, Any]:
    # txt 안에 python dict literal이 들어있는 포맷(인재상/JD 파일)
    return ast.literal_eval(read_text(path))

def sent_simplify(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

def split_bullets(text: str) -> List[str]:
    # 불릿/줄바꿈/쉼표 단위로 가볍게 분할
    parts = re.split(r"[•\-\n]+|,\s", text)
    parts = [sent_simplify(p) for p in parts if sent_simplify(p)]
    return parts

In [ ]:
@dataclass
class Doc:
    id: str
    corpus: str              # resume | question | jd | values
    text: str
    meta: Dict[str, Any]

In [ ]:
def load_resumes(resume_dir: str) -> List[Doc]:
    """
    자소서*_cleaned.txt : JSON 구조(프로젝트/스킬/성과/갈등)
    - 프로젝트/성과/갈등은 '유닛'으로 저장
    - 스킬은 개별 스킬을 한 줄 요약으로 저장
    """
    docs = []
    files = sorted(glob.glob(os.path.join(resume_dir, "*_cleaned.txt")))
    for fp in files:
        data = json.loads(read_text(fp))
        user_tag = os.path.basename(fp).replace("_cleaned.txt", "")
        # 프로젝트
        for p in data.get("projects", []):
            text = f"[프로젝트] {p.get('title','')}: {p.get('summary','')}"
            meta = {
                "user": user_tag, "type": "project",
                "role": p.get("role"), "domain": p.get("domain"),
                "org": p.get("org"), "period": f"{p.get('start')}~{p.get('end')}",
                "tools": p.get("tools"), "techniques": p.get("techniques"),
                "evidence_span": p.get("evidence_span")
            }
            docs.append(Doc(id=str(uuid.uuid4()), corpus="resume", text=sent_simplify(text), meta=meta))
        # 성과
        for a in data.get("achievements", []):
            text = f"[성과] {a.get('ref_title') or '개인 성과'}: {a.get('metric_name')}={a.get('value')}, 임팩트: {a.get('business_or_clinical_impact')}"
            meta = {"user": user_tag, "type": "achievement"}
            docs.append(Doc(id=str(uuid.uuid4()), corpus="resume", text=sent_simplify(text), meta=meta))
        # 갈등
        for c in data.get("conflicts", []):
            text = f"[갈등] 상황:{c.get('context')} 조치:{', '.join(c.get('actions',[]))} 결과:{c.get('outcome')} 교훈:{c.get('lessons')}"
            meta = {"user": user_tag, "type": "conflict", "ref_title": c.get("ref_title")}
            docs.append(Doc(id=str(uuid.uuid4()), corpus="resume", text=sent_simplify(text), meta=meta))
        # 스킬
        for s in data.get("skills", []):
            text = f"[스킬] {s.get('name')} ({s.get('category')})"
            meta = {"user": user_tag, "type": "skill", "category": s.get("category")}
            docs.append(Doc(id=str(uuid.uuid4()), corpus="resume", text=sent_simplify(text), meta=meta))
    return docs

def load_questions(q_dir: str) -> List[Doc]:
    """각 CSV에 question 문자열 1컬럼 존재(파일명으로 카테고리 추출)"""
    docs = []
    csvs = sorted(glob.glob(os.path.join(q_dir, "*.csv")))
    for fp in csvs:
        df = pd.read_csv(fp)
        # 컬럼명 자동 탐지
        col_candidates = [c for c in df.columns if "question" in c.lower()]
        if not col_candidates:
            raise ValueError(f"질문 컬럼을 찾을 수 없습니다: {fp}, columns={df.columns.tolist()}")
        col = col_candidates[0]
        cat = os.path.basename(fp).replace("_unique_questions.csv","").replace(".csv","")
        for q in df[col].fillna("").astype(str).tolist():
            q = sent_simplify(q)
            if not q:
                continue
            meta = {"category": cat}
            docs.append(Doc(id=str(uuid.uuid4()), corpus="question", text=q, meta=meta))
    return docs

def load_values(values_path: str) -> List[Doc]:
    """
    인재상.txt : {회사: {'키워드': [...], '인재상': {슬로건: 설명...}}}
    → 행동지표형 문장으로 소폭 확장
    """
    d = literal_dict_from_txt(values_path)
    docs = []
    for company, blob in d.items():
        keywords = blob.get("키워드", [])
        for slogan, desc in blob.get("인재상", {}).items():
            behaviors = [
                f"{company}의 '{slogan}'을 보여준 사례를 말해달라: {desc}",
                f"{company} 가치 '{slogan}'에 부합하는 행동 기준을 충족한 경험(주도성/협업/문제해결 등)을 구체적으로 설명"
            ]
            for b in behaviors:
                meta = {"company": company, "slogan": slogan, "keywords": keywords}
                docs.append(Doc(id=str(uuid.uuid4()), corpus="values", text=sent_simplify(b), meta=meta))
    return docs

def load_jd(jd_dir: str) -> List[Doc]:
    """
    각 JD txt: {포지션: {'담당 업무': '...', '자격요건': '...', '우대사항': '...'}}
    → 불릿/문장 단위 유닛화
    """
    docs = []
    jds = sorted(glob.glob(os.path.join(jd_dir, "*.txt")))
    for fp in jds:
        comp_hint = os.path.basename(fp).split("_")[0]  # Carrot_JD.txt → 'Carrot'
        d = literal_dict_from_txt(fp)
        for role, spec in d.items():
            for section in ["담당 업무", "자격요건", "우대사항"]:
                content = spec.get(section, "")
                for line in split_bullets(content):
                    if not line:
                        continue
                    meta = {"company_hint": comp_hint, "role": role, "section": section}
                    docs.append(Doc(id=str(uuid.uuid4()), corpus="jd", text=sent_simplify(line), meta=meta))
    return docs

In [ ]:
class VectorStore:
    def __init__(self, model_name: str = "BAAI/bge-m3"):
        self.model = SentenceTransformer(model_name)
        self.indexes: Dict[str, faiss.Index] = {}
        self.metadatas: Dict[str, List[Dict[str, Any]]] = {}

    def _embed(self, texts: List[str]) -> np.ndarray:
        vecs = self.model.encode(
            texts, batch_size=64, show_progress_bar=True, normalize_embeddings=True
        )
        return np.asarray(vecs, dtype="float32")

    def build(self, corpus_name: str, docs: List[Doc]):
        texts = [d.text for d in docs]
        metas = [ {"id": d.id, **d.meta, "corpus":d.corpus, "text": d.text} for d in docs ]
        vecs = self._embed(texts)

        index = faiss.IndexFlatIP(vecs.shape[1])  # cosine(=dot after L2-norm)
        index.add(vecs)

        self.indexes[corpus_name]   = index
        self.metadatas[corpus_name] = metas
        print(f"[{corpus_name}] vectors: {index.ntotal}")

    def save(self, corpus_name: str, out_dir: str = OUT_DIR):
        os.makedirs(out_dir, exist_ok=True)
        faiss.write_index(self.indexes[corpus_name], os.path.join(out_dir, f"{corpus_name}.faiss"))
        with open(os.path.join(out_dir, f"{corpus_name}.meta.json"), "w", encoding="utf-8") as f:
            json.dump(self.metadatas[corpus_name], f, ensure_ascii=False, indent=2)

    def load(self, corpus_name: str, out_dir: str = OUT_DIR):
        self.indexes[corpus_name] = faiss.read_index(os.path.join(out_dir, f"{corpus_name}.faiss"))
        with open(os.path.join(out_dir, f"{corpus_name}.meta.json"), "r", encoding="utf-8") as f:
            self.metadatas[corpus_name] = json.load(f)

    def search(
        self, corpus_name: str, queries: List[str], top_k: int = 10,
        filters: Optional[Dict[str, Any]] = None,
    ) -> List[List[Dict[str, Any]]]:
        """
        filters 예시:
          - {"company": "당근마켓"} (values)
          - {"company_hint": "Carrot"} (jd)
          - {"user": "자소서1"} (resume)
          - {"category": "ICT"} (question)
        """
        # 인덱스/메타 로드
        if corpus_name not in self.indexes:
            self.load(corpus_name)

        metas = self.metadatas[corpus_name]

        # 메타 pre-filter (부분 일치 허용)
        mask = np.array([True]*len(metas))
        if filters:
            keys = list(filters.keys())
            for i, m in enumerate(metas):
                for k in keys:
                    if k not in m:
                        mask[i] = False; break
                    want = str(filters[k]); have = str(m.get(k,""))
                    if fuzz.partial_ratio(want, have) < 80:
                        mask[i] = False; break

        sub_idx = np.where(mask)[0]
        if len(sub_idx) == 0:
            return [[] for _ in queries]

        sub_metas = [metas[i] for i in sub_idx]

        # 서브 인덱스(온더플라이)
        sub_vecs = self._embed([m["text"] for m in sub_metas])
        sub_index = faiss.IndexFlatIP(sub_vecs.shape[1])
        sub_index.add(sub_vecs)

        q_vecs = self._embed(queries)
        D, I = sub_index.search(q_vecs, min(top_k, len(sub_metas)))

        results = []
        for r_i in range(len(queries)):
            items = []
            for j, score in zip(I[r_i], D[r_i]):
                if j == -1:
                    continue
                m = sub_metas[int(j)]
                items.append({"score": float(score), **m})
            results.append(items)
        return results

In [ ]:
import glob, unicodedata, json, ast, uuid

def find_values_path(root="/content/data"):
    # 1) 간단 패턴 탐색
    cands = glob.glob(os.path.join(root, "*재상*.txt"))
    if cands:
        return cands[0]
    # 2) 한글 정규화 비교
    target = unicodedata.normalize("NFC", "인재상.txt")
    for name in os.listdir(root):
        if unicodedata.normalize("NFC", name) == target:
            return os.path.join(root, name)
    return None

def load_values_safe(values_path: str) -> List[Doc]:
    raw = read_text(values_path)
    docs: List[Doc] = []
    # 1) JSON 시도
    try:
        d = json.loads(raw)
    except Exception:
        # 2) Python dict literal 시도
        try:
            d = ast.literal_eval(raw)
        except Exception:
            # 3) 일반 텍스트 라인 처리
            for line in [sent_simplify(x) for x in raw.splitlines() if sent_simplify(x)]:
                docs.append(Doc(id=str(uuid.uuid4()), corpus="values", text=line, meta={"format":"text"}))
            return docs

    # dict 구조 처리
    for company, blob in d.items():
        keywords = blob.get("키워드", [])
        ideals = blob.get("인재상", {}) or {}
        for slogan, desc in ideals.items():
            behaviors = [
                f"{company}의 '{slogan}'을(를) 보여준 실제 사례를 설명: {desc}",
                f"{company} 가치 '{slogan}'에 부합하는 행동 기준을 충족한 경험을 구체적으로 말해달라"
            ]
            for b in behaviors:
                docs.append(Doc(
                    id=str(uuid.uuid4()),
                    corpus="values",
                    text=sent_simplify(b),
                    meta={"company": company, "slogan": slogan, "keywords": keywords}
                ))
    return docs

In [ ]:
# === 데이터 로딩 ===
resume_docs   = load_resumes(os.path.join(DATA_ROOT, "resumes"))
question_docs = load_questions(os.path.join(DATA_ROOT, "questions"))

values_path = find_values_path(DATA_ROOT)
if values_path is None:
    raise FileNotFoundError("⚠️ DATA_ROOT에서 인재상 파일을 찾을 수 없습니다. 파일명을 확인하세요.")
values_docs   = load_values_safe(values_path)

jd_docs       = load_jd(os.path.join(DATA_ROOT, "jd"))

print(f"Loaded counts -> resume={len(resume_docs)}, question={len(question_docs)}, values={len(values_docs)}, jd={len(jd_docs)}")
print("values_path ->", values_path)

# === 인덱스 빌드/저장 ===
vs = VectorStore(model_name="BAAI/bge-m3")
vs.build("resume",   resume_docs)
vs.build("question", question_docs)
vs.build("values",   values_docs)
vs.build("jd",       jd_docs)

for name in ["resume","question","values","jd"]:
    vs.save(name, OUT_DIR)

print("==> 인덱스/메타 저장 완료:", OUT_DIR)

Loaded counts -> resume=80, question=3708, values=116, jd=5903
values_path -> /content/data/인재상.txt


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

[question] vectors: 3708


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

[values] vectors: 116


Batches:   0%|          | 0/93 [00:00<?, ?it/s]

[jd] vectors: 5903
==> 인덱스/메타 저장 완료: /content/artifacts


In [ ]:
# (A) 특정 회사 힌트로 JD 검색
example_queries = ["데이터 분석 관련 핵심 역량"]
results = vs.search("jd", example_queries, top_k=5, filters={"company_hint": "Carrot"})
for r in results[0]:
    print(f"[{r['company_hint']} | {r['role']} | {r['section']}] {r['text']}  (score={r['score']:.3f})")

# (B) 질문 DB에서 ICT 카테고리 상위 유사 문항
example_queries = ["머신러닝 프로젝트에서 모델 선택과 튜닝 경험"]
results = vs.search("question", example_queries, top_k=5, filters={"category":"ICT"})
for r in results[0]:
    print(f"[{r['category']}] {r['text']}  (score={r['score']:.3f})")

# (C) 회사 인재상과 맞닿는 가치 문장
example_queries = ["주도적으로 개선안을 만들고 빠르게 실험한 경험"]
results = vs.search("values", example_queries, top_k=5, filters={"company":"당근마켓"})
for r in results[0]:
    print(f"[{r['company']} | {r['slogan']}] {r['text']}  (score={r['score']:.3f})")

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Carrot | Product Manager - 광고 상품 | 우대사항] 비즈니스 데이터 분석 역량  (score=0.826)
[Carrot | Data Analyst - 광고 | 자격요건] 광고 비즈니스 성장 전략 제시를 위한 데이터 분석 주도 능력  (score=0.682)
[Carrot | Operations Manager - 중고거래 | 우대사항] 분석 및 해석 능력  (score=0.678)
[Carrot | Data Analyst - 광고 | 자격요건] 데이터 기반 인사이트 효과적인 전달 및 소통 능력  (score=0.671)
[Carrot | Operations Manager - 중고거래 | 자격요건] 서비스 리스크 사전 파악 및 데이터 기반 문제 분석 및 선제적 대응 능력  (score=0.669)


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[ICT] 과거 프로젝트를 수행해 본 경험이 있나요 있다면 혹시 그 프로젝트 내에서 어떤 역할을 맡으셨었나요  (score=0.596)
[ICT] 하드웨어 소프트웨어 이에 관한 실무 능력을 갖추기 위해 노력했던 경험에 대해 말씀해 주시겠습니까  (score=0.595)
[ICT] 이전 직장이나 혹은 대학교에서 본인이 해보셨던 프로젝트 가운데에서 생각나는 것이 있다면 한 번 얘기해줘  (score=0.595)
[ICT] 개발 관련하여 여러 경험이 있으신데 개발 프로세스에서 어떤 게 제일 재미있었나요  (score=0.594)
[ICT] 개발 관련하여 일을 오래 하셨는데 기술적으로 어려웠던 점이 있으셨다면 무엇이었는지 사례와 함께 말씀해 주시겠습니까  (score=0.570)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[당근마켓 | 주도적으로 일하는 인재] 당근마켓 가치 '주도적으로 일하는 인재'에 부합하는 행동 기준을 충족한 경험을 구체적으로 말해달라  (score=0.587)
[당근마켓 | 실험적이고 결과 지향적인 인재] 당근마켓 가치 '실험적이고 결과 지향적인 인재'에 부합하는 행동 기준을 충족한 경험을 구체적으로 말해달라  (score=0.576)
[당근마켓 | 실험적이고 결과 지향적인 인재] 당근마켓의 '실험적이고 결과 지향적인 인재'을(를) 보여준 실제 사례를 설명: 정답을 찾기 위해 오랜 시간 고민하지 않고, 빠르게 실험하며 사용자를 위한 최적의 해결책을 찾는 인재  (score=0.574)
[당근마켓 | 끊임없이 배우고 성장하는 인재] 당근마켓 가치 '끊임없이 배우고 성장하는 인재'에 부합하는 행동 기준을 충족한 경험을 구체적으로 말해달라  (score=0.566)
[당근마켓 | 즐겁게 몰입하고 크게 성장하는 인재] 당근마켓 가치 '즐겁게 몰입하고 크게 성장하는 인재'에 부합하는 행동 기준을 충족한 경험을 구체적으로 말해달라  (score=0.549)
